# MULTIHEAD ATTENTION

In [1]:
import sys

sys.executable

'C:\\Users\\user\\anaconda3\\python.exe'

In [17]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import math

In [5]:
sequence_length = 4                  # MY NAME IS VISHNU (4 TOKENS)
batch_size = 1
input_dim = 512                       # FOR EVERY WORD INPUT VECTOR DIM HAS 512
d_model = 512                         # OUTPUT OF ATTENTION UNIT
x = torch.randn((batch_size, sequence_length, input_dim))
x

tensor([[[ 0.1458,  0.5448,  0.3171,  ...,  0.7818, -0.3835, -1.2561],
         [-1.3927, -0.5620, -0.4921,  ..., -0.0276,  0.2941, -1.9411],
         [ 1.6130, -0.3110,  0.8778,  ...,  1.2637,  0.3617, -1.2672],
         [ 0.0891,  0.4357, -0.3769,  ..., -0.7683, -0.5691,  1.0808]]])

In [4]:
x.size()                              #([1,4,512])

torch.Size([1, 4, 512])

In [7]:
qkv_layer = nn.Linear(input_dim, 3 * d_model)
qkv_layer

Linear(in_features=512, out_features=1536, bias=True)

In [9]:
qkv = qkv_layer(x)
qkv

tensor([[[-0.6538,  0.0789,  0.8566,  ...,  0.4444,  0.2386,  0.5976],
         [-0.0644,  0.9934, -0.0627,  ..., -0.4559,  0.2525, -0.6323],
         [-0.0769, -0.3066,  0.4232,  ...,  0.6009,  0.4618, -0.8853],
         [ 0.0464, -0.1677,  0.6598,  ...,  0.6227, -0.2876,  0.3636]]],
       grad_fn=<ViewBackward0>)

In [10]:
qkv.shape                           # ([1,4,1536])          512*3 = 1536

torch.Size([1, 4, 1536])

In [12]:
num_heads = 8
head_dim = d_model // num_heads
qkv = qkv.reshape(batch_size, sequence_length, num_heads, 3 * head_dim)
qkv

tensor([[[[-0.6538,  0.0789,  0.8566,  ..., -0.6331,  0.6063, -0.7388],
          [ 0.0882,  0.0067,  0.2405,  ..., -0.1896,  0.0159,  0.8613],
          [-0.0174, -0.9481,  0.4832,  ...,  0.8955, -0.1091, -0.2535],
          ...,
          [-0.3506,  0.0642, -1.1343,  ..., -0.9535,  0.0884,  0.1644],
          [-0.2446, -0.5925,  0.5105,  ..., -0.3860,  0.6691,  0.3578],
          [-0.4290,  0.5060,  0.2739,  ...,  0.4444,  0.2386,  0.5976]],

         [[-0.0644,  0.9934, -0.0627,  ...,  0.7971,  0.3028, -0.0422],
          [ 0.4334, -0.3570,  0.2367,  ..., -0.3192,  0.2682,  1.3620],
          [-0.3155, -0.2850, -1.2711,  ...,  0.0703,  0.6375,  0.5030],
          ...,
          [ 0.3730, -0.4761,  0.6795,  ..., -0.5216, -0.5191,  0.8454],
          [-0.1654, -0.7986,  0.4173,  ..., -0.5151,  0.4683, -0.1285],
          [-0.7864, -0.0896, -0.6988,  ..., -0.4559,  0.2525, -0.6323]],

         [[-0.0769, -0.3066,  0.4232,  ...,  0.6631,  0.9911,  0.5760],
          [ 0.4077, -0.8850,  

In [13]:
qkv.shape                         # ([1,4,8,192])

torch.Size([1, 4, 8, 192])

In [14]:
# to perform parellel last 2 operations

qkv.permute(0,2,1,3)              # ([batch_size, num_head, sequence_length, 3 * head_dim ])
qkv.shape                          # ([1,8,4,192])    

torch.Size([1, 4, 8, 192])

In [15]:
# chunking into a q , k, v

q,k,v = qkv.chunk(3, dim= -1)
q.shape,k.shape,v.shape        # we get for q k v   ([1,8,4,64])     192/3 = 64

(torch.Size([1, 4, 8, 64]),
 torch.Size([1, 4, 8, 64]),
 torch.Size([1, 4, 8, 64]))

## Self Attention for Multiple heads

In [18]:
d_k = q.size()[-1]
scaled = torch.matmul(q, k.transpose(-2, -1))/ math.sqrt(d_k)       # if q (4*64) k is (64*4)      # these are tensors we cant use k.T
scaled.shape                                                            # ([1,8,4,4])

torch.Size([1, 4, 8, 8])

### MASKING

In [21]:
mask = torch.full(scaled.size(), float('-inf'))
mask = torch.triu(mask, diagonal=1)                # make upper triangle infinity
mask[0][1]                                         #mask for input of single data

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

(scaled +mask)[0][0]

In [23]:
scaled = scaled + mask                         # help us to understand the words 
scaled

tensor([[[[ 0.0035,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,
              -inf],
          [ 0.2269, -0.2004,    -inf,    -inf,    -inf,    -inf,    -inf,
              -inf],
          [ 0.0602, -0.0716, -0.0345,    -inf,    -inf,    -inf,    -inf,
              -inf],
          [-0.2234, -0.4465, -0.0787,  0.2251,    -inf,    -inf,    -inf,
              -inf],
          [ 0.0561, -0.0220, -0.4301,  0.0745,  0.1796,    -inf,    -inf,
              -inf],
          [-0.5520,  0.2032, -0.0732, -0.4269, -0.4371, -0.1512,    -inf,
              -inf],
          [ 0.0484,  0.4813, -0.0359,  0.1483,  0.0365, -0.5418,  0.4600,
              -inf],
          [-0.1119,  0.0035, -0.0086,  0.2691, -0.0194,  0.0714, -0.1101,
           -0.0227]],

         [[-0.2807,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,
              -inf],
          [-0.1885, -0.1245,    -inf,    -inf,    -inf,    -inf,    -inf,
              -inf],
          [-0.2620,  0.4379, -0.0949,    -inf,  

In [25]:
attention = F.softmax(scaled, dim = -1)


In [26]:
attention[0][0]                         # make upper triangle with zero now

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.6052, 0.3948, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3589, 0.3146, 0.3265, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2212, 0.1769, 0.2556, 0.3463, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2132, 0.1972, 0.1311, 0.2172, 0.2413, 0.0000, 0.0000, 0.0000],
        [0.1178, 0.2506, 0.1901, 0.1335, 0.1321, 0.1759, 0.0000, 0.0000],
        [0.1313, 0.2024, 0.1207, 0.1451, 0.1297, 0.0728, 0.1981, 0.0000],
        [0.1100, 0.1235, 0.1220, 0.1611, 0.1207, 0.1322, 0.1102, 0.1203]],
       grad_fn=<SelectBackward0>)

In [27]:
values = torch.matmul(attention, v)
values.shape                                #([1,8,4,64])

torch.Size([1, 4, 8, 64])

In [31]:
# lets create a functiom for this , we dont know we are dealing with encoder or decoder

import math

def scaled_dot_product(q,k,v, mask = None):
    d_k = q.size()[-1]
    scaled = torch.matmul(q, k.transpose(-1,-2)) /math.sqrt(d_k)
    if mask is not None:
        scaled =scaled + mask
    attention = F.softmax(scaled, dim = -1)
    values = torch.matmul(attention, v)
    return values, attention

In [32]:
values, attention = scaled_dot_product(q, k, v, mask = mask)

In [35]:
attention.shape                  #([1,8,4,4])
print(attention[0][0])
values.size()                     #([1,8,4,64])



tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.6052, 0.3948, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3589, 0.3146, 0.3265, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2212, 0.1769, 0.2556, 0.3463, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2132, 0.1972, 0.1311, 0.2172, 0.2413, 0.0000, 0.0000, 0.0000],
        [0.1178, 0.2506, 0.1901, 0.1335, 0.1321, 0.1759, 0.0000, 0.0000],
        [0.1313, 0.2024, 0.1207, 0.1451, 0.1297, 0.0728, 0.1981, 0.0000],
        [0.1100, 0.1235, 0.1220, 0.1611, 0.1207, 0.1322, 0.1102, 0.1203]],
       grad_fn=<SelectBackward0>)


torch.Size([1, 4, 8, 64])

In [37]:
# now lets concat each heads 

values = values.reshape(batch_size, sequence_length, num_heads * head_dim)
values.size()                                    #([1,4,512])

torch.Size([1, 4, 512])

In [38]:
# for communction eachother to share information create NN
linear_layer = nn.Linear(d_model, d_model)             #(512,512)

In [39]:
out = linear_layer(values)
out.shape                                      # ([1,4,512])

torch.Size([1, 4, 512])

In [ ]:
#lets create all in one set 

In [14]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import math


def scaled_dot_product(q,k,v, mask = None):
    d_k = q.size()[-1]
    scaled = torch.matmul(q, k.transpose(-1,-2)) /math.sqrt(d_k)
    if mask is not None:
        scaled =scaled + mask
    attention = F.softmax(scaled, dim = -1)
    values = torch.matmul(attention, v)
    return values, attention

class MultiheadAttention(nn.Module):
    def __init__(self, input_dim, d_model, num_heads):
        super().__init__()
        self.input_dim = input_dim
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads
        self.qkv_layer = nn.Linear(input_dim, 3*d_model)
        self.linear_layer = nn.Linear(d_model, d_model)

    def forward(self, x, mask = None):
        batch_size ,sequence_length, input_dim = x.size()
        print(f"x.size():{x.size()}")
        qkv = self.qkv_layer(x)
        print(f"qkv.size(): {qkv.size()}")
        qkv = qkv.reshape(batch_size, sequence_length, self.num_heads, 3* self.head_dim)
        print(f"qkv.size(): {qkv.size()}")
        qkv = qkv.permute(0,2,1,3)
        print(f"qkv.size(): {qkv.size()}")
        q, k, v = qkv.chunk(3, dim = -1)
        print(f"q size {q.size()}\nk size {k.size()}\nv size {v.size()}")


        values , attention = scaled_dot_product(q, k, v, mask)
        print(f"values size {values.size()}\nattention.size {attention.size()}")
        values = values.reshape(batch_size, seq_len, self.num_heads * self.head_dim)
        print(f"values size {values.size()}")
        out = self.linear_layer(values)
        print(f"out size {out.size()}")

        return out

input_dim = 512
d_model = 512
num_heads= 8

batch_size = 30
seq_len= 5
x = torch.randn((batch_size, seq_len, input_dim))

model = MultiheadAttention(input_dim, d_model, num_heads)
out = model.forward(x)    
        
        
        

x.size():torch.Size([30, 5, 512])
qkv.size(): torch.Size([30, 5, 1536])
qkv.size(): torch.Size([30, 5, 8, 192])
qkv.size(): torch.Size([30, 8, 5, 192])
q size torch.Size([30, 8, 5, 64])
k size torch.Size([30, 8, 5, 64])
v size torch.Size([30, 8, 5, 64])
values size torch.Size([30, 8, 5, 64])
attention.size torch.Size([30, 8, 5, 5])
values size torch.Size([30, 5, 512])
out size torch.Size([30, 5, 512])
